In [1]:
import numpy as np
import pandas as pd
import os
import math
import random
import shutil
from PIL import Image
from skimage.io import imread

In [2]:
path=r'C:\Users\Biswarup Mistry\Desktop\USC_SIPI_Databases' #USC_SIPI_Database directory [To Change 1]
dir_list=os.listdir(path)
print(len(dir_list))

4


In [3]:
for item in dir_list :
    k=str(path+'\\'+str(item))
    #img = imread(k)
    dir_lst=os.listdir(k)
    print(len(dir_lst))
    print('For Training Dataset  : ',math.floor((len(dir_lst)*90)/100))

2
For Training Dataset  :  1
39
For Training Dataset  :  35
2
For Training Dataset  :  1
2
For Training Dataset  :  1


In [4]:
a=math.floor((38*90)/100)
print(a)

34


In [5]:
#random.choices(list, k=3) #Choose multiple random items from a list, set, or any data structure. 

In [6]:
directory = "Training Set"
directory1 = "Test Set"

In [7]:
for item in dir_list :
    if item != "misc" :
        p=str(path+'\\'+str(item))
        dir_lst=os.listdir(p)
        #a=random.choices(dir_lst, k=3)  #Choose multiple random items from a list, set, or any data structure.
        i=1
        cal=math.floor((len(dir_lst)*90)/100)
        cal1=len(dir_lst)-cal
        #directory = "Training Set"
        parent_dir = p
        path1 = os.path.join(parent_dir, directory)
        if os.path.exists(path1)==False :
            os.mkdir(path1) #making the 'Training Set' Directory
            new_path=str(parent_dir+'\\'+directory)

            while i<=cal :
                a=random.choices(os.listdir(p)) #randomly choosen 80% training data set of current dataset & gives a list
                for image in a :
                    if image != directory :
                        old_path=p+'\\'+image
                        n_path=new_path+'\\'+image
                        #try :
                        shutil.move(old_path, n_path) #to move the files
                        #except FileNotFoundError :
                            #print('Error : File not Found for Image = ',image)
                        i+=1

            #directory1 = "Test Set"
            #if os.path.exists(path1)==False :
            parent_dir = p
            path1 = os.path.join(parent_dir, directory1)
            os.mkdir(path1) #making the 'Training Set' Directory
            new_path=str(parent_dir+'\\'+directory1)
            a=os.listdir(p)
            for item in a:
                if (item != directory) and (item != directory1) :
                    old_path=p+'\\'+item
                    n_path=new_path+'\\'+item
                    shutil.move(old_path, n_path) 


In [8]:
def get_Noise(image, prob):
    #'''
    #Add salt and pepper noise to image
    #prob: Probability of the noise
    #'''
    output = image.copy()
    if len(image.shape) == 2:
        black = 0
        white = 255            
    else:
        colorspace = image.shape[2]
        if colorspace == 3:  # RGB
            black = np.array([0, 0, 0], dtype='uint8')
            white = np.array([255, 255, 255], dtype='uint8')
        else:  # RGBA
            black = np.array([0, 0, 0, 255], dtype='uint8')
            white = np.array([255, 255, 255, 255], dtype='uint8')
    probs = np.random.random(output.shape[:2])
    output[probs < (prob / 2)] = black
    output[probs > 1 - (prob / 2)] = white
    return output

In [9]:
def PSNRVsNoiseDensity(dirc_list,dataframe): #getting corresponding mse(s) for given noise density 'd'
    r=255.0
    psnr='PSNR_of_90_percent_Data.csv'
    ssim1='SSIM_of_90_percent_Data.csv'
    #dataframe = pd.DataFrame()
    for item in os.listdir(dirc_list) :
        if item!=psnr and item!=ssim1:
            #k=str(path+str(item))
            k=str(dirc_list+'\\'+item)
            img = imread(k)
            x=[]
            y=[]
            df=np.array([])
            #i=iteration
            #d=0.0
            #while(i>0 and d<=0.3):
            #D=[0.0]
            D=[0.1,0.25,0.3,0.35,0.4]
            #D=[0.1,0.2,0.3]
            for d in D:
                noise_img=get_Noise(img,d) #getting noisy image
                mse=np.square(np.subtract(img,noise_img)).mean() #calculating MSE of Original image and Image with Noise
                psnr=10*np.log10(np.square(r)/mse) #getting the psnr of Original image and Noisy image
                x.append(d)
                y.append(psnr)
                df=np.append(df,psnr)
                #d=d+0.1
                #i=i-1
            dataframe[item] = df
        #dataframe.to_csv('PSNRvsNoiseDensity.csv', index=False) #creating the PSNR valu csv file


In [10]:
def createFolder(path):
    count=0
    count1=0
    dir_list=os.listdir(path)
    parent_dir=path
    dir_list=os.listdir(path)
    check1=str(path+'\\'+'256')
    check2=str(path+'\\'+'512')
    check3=str(path+'\\'+'1024')
    
    isExist1=os.path.exists(check1)
    isExist2=os.path.exists(check2)
    isExist3=os.path.exists(check3)
    
    if isExist1==isExist2==isExist3==False:
        for item in dir_list:
            if (item!=check1) and (item!=check2) and (item!=check3):
                k=str(path+'\\'+item)
                img = Image.open(k)
                w, h= img.size
                img.close()
                file_path=str(path+'\\'+str(w))
                isExist=os.path.exists(file_path)
                if isExist==False:
                    directory=str(w)
                    path1 = os.path.join(parent_dir, directory)
                    os.mkdir(path1)
                    count1+=1
    else:
        print('Folders already exist!')
    print('Created Folder : ',count1)
    

In [11]:
#moving images according to their image size to appropriate folders

def moveImage(path):
    psnr='PSNR_of_90_percent_Data.csv'
    ssim1='SSIM_of_90_percent_Data.csv'
    count=0
    dir_list=os.listdir(path)
    check1=str(path+'\\'+'256')
    check2=str(path+'\\'+'512')
    check3=str(path+'\\'+'1024')
    #print(len(dir_list))
    if len(dir_list)>3:
        for item in dir_list:
            k=str(path+'\\'+item) 
            isdir = os.path.isdir(k)
            #if isdir== False and item!=psnr and item!=ssim1  :
            if item.lower().endswith(('.png', '.jpg', '.jpeg','.tiff')) :
                img = Image.open(k)
                w, h= img.size
                file_path=str(path+'\\'+str(w))
                old_path=str(path+'\\'+str(item))
                n_path=str(file_path+'\\'+str(item))
                img.close()
                shutil.move(old_path, n_path)
                count+=1
        
        print('Image(s) moved : ',count)
    elif len(dir_list)==3:
        print('Nothing to Move!')


In [12]:
psnr='PSNR_of_90_percent_Data.csv'
ssim1='SSIM_of_90_percent_Data.csv'
for item in dir_list :
    if item != "misc" :
        p=str(path+'\\'+str(item))
        dir_lst=os.listdir(p)
        for item in dir_lst:
            if item != directory1 :
                new_path=str(p+'\\'+str(item))
                createFolder(new_path)
                moveImage(new_path)
                for item in os.listdir(new_path):
                    p1=str(new_path+'\\'+str(item))
                    if os.path.exists(str(p1+'\\'+psnr))==False:
                        dataframe = pd.DataFrame()
                        PSNRVsNoiseDensity(p1,dataframe)
                        #for item in os.listdir(new_path): #under Training Set
                        #    k=str(new_path+'\\'+str(item))
                        #    getPSNR(image)
                        dataframe.to_csv(p1+'\\'+'PSNR_of_90_percent_Data.csv', index=False) 
                    else :
                        print(psnr,' Exists!!')
            #print(dir_lst)

Created Folder :  2
Image(s) moved :  34
Created Folder :  2
Image(s) moved :  62
Created Folder :  2
Image(s) moved :  57


In [13]:
for item in os.listdir(path):
    if item != "misc":
        p=str(path+'\\'+str(item))
        
        for item in os.listdir(p):
            if item == directory :
                print(item)
                new_path=str(p+'\\'+directory) #under training set
                for item in os.listdir(new_path):
                    k=str(new_path+'\\'+str(item))
                    print(k)
                    try:
                        data=pd.read_csv(k+'\\'+'PSNR_of_90_percent_Data.csv')
                        print(data)
                    except:
                        print("PSNR_of_90_percent_Data.csv not Exists!!")


Training Set
C:\Users\Biswarup Mistry\Desktop\USC_SIPI_Databases\aerials\Training Set\1024
   2.2.01.tiff  2.2.02.tiff  2.2.05.tiff  2.2.06.tiff  2.2.07.tiff  \
0    37.939583    37.377952    37.848184    37.677826    37.632684   
1    33.964263    33.414200    33.867855    33.694090    33.651551   
2    33.179290    32.614951    33.074967    32.900683    32.850298   
3    32.515860    31.945373    32.399274    32.224959    32.183121   
4    31.923704    31.375238    31.821997    31.646585    31.594801   

   2.2.08.tiff  2.2.09.tiff  2.2.10.tiff  2.2.11.tiff  2.2.12.tiff  ...  \
0    37.839040    37.631130    37.644638    37.895194    37.946339  ...   
1    33.890177    33.659488    33.666624    33.874226    33.949239  ...   
2    33.082879    32.866938    32.852205    33.077566    33.160349  ...   
3    32.413994    32.205587    32.193858    32.406225    32.492011  ...   
4    31.841911    31.624506    31.620742    31.837754    31.896200  ...   

   2.2.15.tiff  2.2.16.tiff  2.2.17.t